In [15]:
from bs4 import BeautifulSoup
import pandas as pd
from copy import deepcopy

In [16]:
def is_valid_html(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    tags = [tag.name for tag in soup.find_all()]
    valid = len(tags) > 0
    if not valid:
        print(html_string)
    return valid

In [17]:
def get_tag_names(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    tags = [tag.name for tag in soup.find_all()]
    return tags

In [18]:
def get_tags(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    tags = [tag for tag in soup.find_all()]
    return tags

In [19]:
def get_attrs(html_tags):
    attrs = []
    for tag in html_tags:
        for key in tag.attrs.keys():
            if key not in attrs:
                attrs.append(key)
    return attrs


In [20]:
def get_attr_values(html_tags):
    attr_values = []
    for tag in html_tags:
        for key in tag.attrs.keys():
            if tag.attrs[key] not in attr_values:
                attr_values.append(tag.attrs[key]) 
    return attr_values

In [21]:
def has_required_attr_vals(html_attr_vals):
    if "description" not in html_attr_vals:
        return False
    return True

In [22]:
def remove_cprcode_from_description(df1):
    df = deepcopy(df1)
    for cprcode in df.cprcode:
        index = df[df["cprcode"] == cprcode].index.item()
        cpr_string = ("0" * (7 - len(str(cprcode)))) + str(cprcode)
        title_found = False
        description_found = False
        try:
            newSeoOutput = df[df["cprcode"] == cprcode].seoOutputText.item().replace(cpr_string, "")
            newSeoOutput = newSeoOutput.replace(str(cprcode), "")
            if "Title: " in newSeoOutput:
                # print(df[df["cprcode"] == cprcode].seoOutputText.item())
                title_found = True
                newSeoOutput = newSeoOutput.replace("Title: ", "<title>")
                
            if "Description: " in newSeoOutput:
                description_found = True
                newSeoOutput = newSeoOutput.replace("Description:", '</title><meta name="description" content="')
            if title_found and description_found:
                newSeoOutput = newSeoOutput + '">'
            elif title_found:
                newSeoOutput = newSeoOutput + '</title>'
            df.loc[index, "seoOutputText"] = newSeoOutput
            if title_found:
                # print(df[df["cprcode"] == cprcode].seoOutputText.item())
                pass
        except Exception as e:
            print("error: ", e)
        
    
    return df

In [23]:
df = pd.read_csv(f"dynamodb_export_full_output_{0}.csv")
df = remove_cprcode_from_description(df)
df

,cprcode,pr_active,pr_engname,pr_country_en,pr_keyword_en,pr_filter_th,pr_filter_en,online_category_l1_th,online_category_l1_en,online_category_l2_th,...,meta_keywords,meta_description,product_attribute,translated_keyword,translated_name1,translated_name2,translated_name3,translated_content,seoInputText,seoOutputText
0,225407,True,KONJAC LINGUINI,TH,NaN,NaN,NaN,อาหารสำเร็จรูปและไข่,Delicatessen & Eggs,อาหารพร้อมทาน,...,"Delicatessen & Eggs,Asian Deli,Noodles,Moku,Ko...",NaN,NaN,Into,Invade the Romoku brand,Invade the Romoku brand,Invade the Romoku brand,Invade the Romoku brand,"please make a seo tag for this product, includ...",<title>KONJAC LINGUINI - Moku Konjac Flat Nood...
1,62644,True,STUTE APRICOT JAM 430 G.,DE,NaN,"Grocery,Low Sugar Sugar Free,Health,Jam Honey...","Grocery,Low Sugar Sugar Free,Health,Jam Honey...",สินค้าบริโภค,Grocery,แยม น้ำผึ้ง และ สเปรดทาขนมปัง,...,"Grocery,Jam Honey & Spreads,Jams & Marmalades,...",Stute Foods' extensive range of 'No Sugar Adde...,"{'origin': '', 'description': ""Stute Foods' ex...",Into,Mare apricot jam 430 G.,Apricot Jam,Mare apricot jam 430g,Mare apricot jam 430g,"please make a seo tag for this product, includ...",<title>Stute Apricot Jam 430g - Low Sugar & Su...
2,192167,True,COLGATE TOTAL CHARCOAL DEEP CLEAN 150G,TH,NaN,"Dental Care,Non-Food","Dental Care,Non-Food",สุขภาพและความงาม,Health & Beauty,ผลิตภัณฑ์ดูแลเหงือกและฟัน,...,"Health & Beauty,Dental Care,Toothpaste,Colgate...",NaN,NaN,Into,"Colgate, Total Charcoal, Clean 150 grams",Total Charcoal Deep Clean,Colgate Total Charcoal Deep Clean 150g,Colgate Total Charcoal Deep Clean 150g,"please make a seo tag for this product, includ...",<title>COLGATE TOTAL CHARCOAL DEEP CLEAN 150G ...
3,243619,True,DOUGLAS-CT1725-AMBER FOX,CH,NaN,NaN,NaN,เครื่องใช้ในครัวเรือนและผลิตภัณฑ์สำหรับสัตว์เล...,Pet & Household,ของเล่น,...,"Pet & Household,Toys,Dolls,Douglas,Amber Fox",NaN,NaN,Into,Douglas -CT1725-Amber Fox Doll,Into,Douglas -CT1725-Amber Fox Doll,Douglas -CT1725-Amber Fox Doll,"please make a seo tag for this product, includ...",<title>DOUGLAS-CT1725-AMBER FOX Doll | Villa M...
4,195886,True,HERR'S MEDIUM SALSA DIP 454 G.,US,NaN,"Condiments and Dressings,Grocery","Condiments and Dressings,Grocery",สินค้าบริโภค,Grocery,เครื่องปรุงรสและน้ำสลัด,...,"Grocery,Condiments & Dressings,Condiments,Herr...",NaN,NaN,Into,Herr's Media Salza Dip Sauce 454 G.,Medium Salsa Dip,Herr's Medium Salsa Dip 454g,Herr's Medium Salsa Dip 454g,"please make a seo tag for this product, includ...",<title>Herr's Medium Salsa Dip 454g - Villa Ma...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,238468,True,WATTIES SPAGHETTI IN TOMATO SAUCE 420G18,NZ,NaN,NaN,NaN,NaN,Grocery,NaN,...,"Grocery,Cans & Packets,Meals & Meats,Wattie's,...",NaN,NaN,Into,Waties Spaghetti Intometo Sauce 420 A. 1811,Wattests Spagetti in Tomato Sauce 420g1,Wattests Spagetti in Tomato Sauce 420g1,Wattests Spagetti in Tomato Sauce 420g1,"please make a seo tag for this product, includ...",<title>Wattie's Spaghetti In Tomato Sauce 420g...
1949,145047,True,HARIBO WINE GUMS JELLY 500 G.,DE,NaN,NaN,NaN,NaN,Confectionery & Snacks,NaN,...,"Confectionery & Snacks,Candies & Marshmallows,...",NaN,NaN,Into,Haribo Gams Jelly 500 grams,Haribo Gams Jelly 500 g.,Haribo Gams Jelly 500 g.,Haribo Gams Jelly 500 g.,"please make a seo tag for this product, includ...",<title>HARIBO Wine Gums Jelly 500g - Buy Onlin...
1950,196184,True,CROISSANT CHOCOLATE BUTTER,TH,Bakery,"Fresh,Bakery Snacks","Fresh,Bakery Snacks",เบเกอรี่,Bakery,ครัวซองต์,...,"Bakery,Croissants,Flavoured,,Croissant Chocola...",NaN,NaN,bakery,CROISSANT CHOCOLATE BUTTER,Croissant Chocolate Butter,Jagota Croissant Chocolate Butter 1 Pk,Jagota Croissant Chocolate Butter 1 PK Bakery,"please make a seo tag for this product, includ...",<title>Jagota Croissant Chocolate Butter 70g |...
1951,232635,True,VISTAMAR SEPIA RT SAUVIGNON BLANC,CL,NaN,NaN,NaN,"เบียร์, ไวน์และสุรา",Beer Wine & Spirits,ไวน์ขาว,...,"Beer Wine 

In [24]:
df.head(10).seoOutputText

0    <title>KONJAC LINGUINI - Moku Konjac Flat Nood...
1    <title>Stute Apricot Jam 430g - Low Sugar & Su...
2    <title>COLGATE TOTAL CHARCOAL DEEP CLEAN 150G ...
3    <title>DOUGLAS-CT1725-AMBER FOX Doll | Villa M...
4    <title>Herr's Medium Salsa Dip 454g - Villa Ma...
5    <title> Sprite Lemon Lime Flavour No Sugar 500...
6    <title>NIVEA DEO DRY COMFORT STICK 40ML | Vill...
7    <title>OGX Conditioner Biotin & Collagen 385ml...
8    <title>Vicenzi Matilde Lemon Cream 150g | Vill...
9    <title>Sloane's Bacon Back 200g - Villa Market...
Name: seoOutputText, dtype: object

In [25]:

df["valid_html"] = df.seoOutputText.apply(is_valid_html)
df["tag_names"] = df.seoOutputText.apply(get_tag_names)
df["tags"] = df.seoOutputText.apply(get_tags)
df["attributes"] = df.tags.apply(get_attrs)
df["attr_values"] = df.tags.apply(get_attr_values)
df["has_required_attr_values"] = df.attr_values.apply(has_required_attr_vals)

In [26]:
df.head(10).tag_names

0          [title, meta]
1    [title, meta, meta]
2    [title, meta, meta]
3    [title, meta, meta]
4    [title, meta, meta]
5    [title, meta, meta]
6    [title, meta, meta]
7    [title, meta, meta]
8                [title]
9    [title, meta, meta]
Name: tag_names, dtype: object

In [27]:
df.head(10).tags

0    [[KONJAC LINGUINI - Moku Konjac Flat Noodle 16...
1    [[Stute Apricot Jam 430g - Low Sugar & Sugar F...
2    [[COLGATE TOTAL CHARCOAL DEEP CLEAN 150G | Vil...
3    [[DOUGLAS-CT1725-AMBER FOX Doll | Villa Market...
4    [[Herr's Medium Salsa Dip 454g - Villa Market]...
5    [[ Sprite Lemon Lime Flavour No Sugar 500ml - ...
6    [[NIVEA DEO DRY COMFORT STICK 40ML | Villa Mar...
7    [[OGX Conditioner Biotin & Collagen 385ml - Vi...
8    [[Vicenzi Matilde Lemon Cream 150g | Villa Mar...
9    [[Sloane's Bacon Back 200g - Villa Market], []...
Name: tags, dtype: object

In [28]:
df.head(10).attributes

0    [name, content]
1    [name, content]
2    [name, content]
3    [name, content]
4    [name, content]
5    [name, content]
6    [name, content]
7    [name, content]
8                 []
9    [name, content]
Name: attributes, dtype: object

In [29]:
df.head(10).attr_values

0    [description, KONJAC LINGUINI is a Moku Konjac...
1    [description, Stute Foods' No Sugar Added Apri...
2    [description, COLGATE TOTAL CHARCOAL DEEP CLEA...
3    [description, Introducing the DOUGLAS-CT1725-A...
4    [description, Herr's Medium Salsa Dip 454g is ...
5    [description,  Sprite Lemon Lime Flavour No Su...
6    [description, Villa Market offers NIVEA DEO DR...
7    [description, OGX Conditioner Biotin & Collage...
8                                                   []
9    [description, Sloane's Bacon Back 200g is a de...
Name: attr_values, dtype: object

In [30]:
df.head(10).has_required_attr_values

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8    False
9     True
Name: has_required_attr_values, dtype: bool

In [31]:
invalid_htmls = 0
doesnt_have_required_attrs = 0
for i in range(10):
    df = pd.read_csv(f"dynamodb_export_full_output_{i}.csv").fillna("")
    df = remove_cprcode_from_description(df)
    df["valid_html"] = df.seoOutputText.apply(is_valid_html)
    df["tag_names"] = df.seoOutputText.apply(get_tag_names)
    df["tags"] = df.seoOutputText.apply(get_tags)
    df["attributes"] = df.tags.apply(get_attrs)
    df["attr_values"] = df.tags.apply(get_attr_values)
    df["has_required_attr_values"] = df.attr_values.apply(has_required_attr_vals)
    invalid_htmls += len(df[df["valid_html"] == False])
    doesnt_have_required_attrs += len(df[df["has_required_attr_values"] == False])
    df.to_csv(f"reformatted_seo_output{i}.csv", index=False)


In [32]:
print("invalid_htmls: ", invalid_htmls)
print("doesnt_have_required_attrs: ", doesnt_have_required_attrs)

invalid_htmls:  8
doesnt_have_required_attrs:  2633


# Merge Files
## Load all 10 files at the same time (from nic)

In [88]:
mergedDf = pd.concat([pd.read_csv(f"reformatted_seo_output{i}.csv") for i in range(10)])
mergedDf.shape

(19530, 44)

## Check for na

In [60]:
import numpy as np
mergedDf[mergedDf.valid_html == False & mergedDf.seoOutputText.isna()].shape

(8, 44)

## Filter out the valid and invalid ones

In [90]:
invalidDf = mergedDf[~mergedDf.has_required_attr_values]
validDf = mergedDf[mergedDf.has_required_attr_values]
invalidDf.attr_values

8                                                      []
29                                                     []
30      ['keywords', 'Haagen-Dazs, Salted Caramel, 500...
34      ['keywords', 'Barilla,Bolognese Pasta Sauce,In...
35                                                     []
                              ...                        
1926                                                   []
1940    ['keywords', "Wall's, Paddle Pop Ice Shot Frui...
1942                                                   []
1948                                                   []
1951                                                   []
Name: attr_values, Length: 2633, dtype: object

In [93]:
generator = (i for i in range(1000))

In [97]:
# just keep running this cell to and item will keep running
i = next(generator)
print('item: ',i)
print("invalid example: ", len(invalidDf))
r = invalidDf.iloc[i].seoOutputText
print(r)
print("valid example:", len(validDf))
r = validDf.iloc[i].seoOutputText
print(r)

item:  3
invalid example:  2633
<title>Barilla Bolognese Pasta Sauce 400g - Villa Market</title><meta name="description" content="Barilla Bolognese Pasta Sauce 400g is a delicious Italian-style pasta sauce made with tomatoes, onions, garlic, and herbs. It is perfect for creating a classic Bolognese sauce for your favorite pasta dishes. This sauce is made with all-natural ingredients and is free from preservatives, artificial colors, and flavors. It is also gluten-free and vegan-friendly. Barilla Bolognese Pasta Sauce is a great way to add flavor and texture to your favorite pasta dishes. It is perfect for creating a classic Bolognese sauce for your favorite pasta dishes. This sauce is made with all-natural ingredients and is free from preservatives, artificial colors, and flavors. It is also gluten-free and vegan-friendly. Barilla Bolognese Pasta Sauce is a great way to add flavor and texture to your favorite pasta dishes. Enjoy the delicious taste of Italy with Barilla Bolognese Pasta